## Setup

In [552]:
#Import Libraries
import pandas as pd
from pathlib import Path

In [553]:
#Declare Filepaths
schoolDataFile = Path("Resources/schools_complete.csv")
studentDataFile = Path("Resources/students_complete.csv")

outFilePath = Path("Output/finishedAnalysis")

In [554]:
#Create main dataframe for analysis by loading the .csv and merging on school_name, displaying the head
schoolData = pd.read_csv(schoolDataFile)
studentData = pd.read_csv(studentDataFile)

fullData = pd.merge(studentData, schoolData, how="left", on=["school_name", "school_name"])
fullData.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

In [555]:
#Unique Schools
schoolCount = fullData["school_name"].nunique()
schoolCount

15

In [556]:
#Total Students
studentCount = fullData['Student ID'].nunique()
studentCount

39170

In [557]:
#Total Budget
totalBudget = schoolData['budget'].sum()
totalBudget

24649428

In [558]:
#Average Math
mathAverage = fullData['math_score'].mean()
mathAverage

78.98537145774827

In [559]:
#Average Reading
readAverage = fullData['reading_score'].mean()
readAverage

81.87784018381414

In [560]:
#Math Passing Count
mminScore = 70
mpassingCount = fullData[(fullData['math_score'] >= mminScore)].count()['student_name']
mpassingPercent = (mpassingCount / studentCount) * 100
mpassingPercent


74.9808526933878

In [561]:
#Reading Passing Count
rminScore = 70
rpassingCount = fullData[(fullData['reading_score'] >= rminScore)].count()['student_name']
rpassingPercent = (rpassingCount / studentCount) * 100
rpassingPercent

85.80546336482001

In [562]:
#Percent passing both math and reading
mminScore = 70
rminScore = 70
mrpassingCount = fullData[(fullData['reading_score'] >= rminScore) & (fullData['math_score'] >= mminScore)].count()['student_name']

mrpassingPercent = (mrpassingCount / studentCount) * 100
mrpassingPercent

65.17232575950983

In [563]:
#Create district summary dataframe, format, and display
summaryDistrict = pd.DataFrame({'Total Schools' : [schoolCount],
                        'Total Students' : [studentCount],
                        'Total Budget' : [totalBudget],
                        'Average Math Score' : [mathAverage],
                        'Average Reading Score' : [readAverage],
                        'Percent Passing Math' : [mpassingPercent],
                        'Percent Passing Reading' : [rpassingPercent],
                        'Percent Passing Overall' : [mrpassingPercent]})

summaryDistrict['Total Students'] = summaryDistrict['Total Students'].map('{:,}'.format)
summaryDistrict['Total Budget'] = summaryDistrict['Total Budget'].map("${:,.2f}".format)

summaryDistrict

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Overall
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [564]:
#Shorthand dataframe merge function for this section
def mergeDataFrames(frame1: pd.DataFrame, frame2: pd.DataFrame, oncol: str):
    return pd.merge(frame1, frame2, how = 'left', on=[oncol])

In [565]:
#Select all school types
schoolTypes = schoolData[['school_name', 'type']]

In [566]:
#Student count per school
perSchoolCount = fullData['school_name'].value_counts()
fullSummary = mergeDataFrames(schoolTypes, perSchoolCount, 'school_name')
fullSummary.rename(columns={'count' : 'Student Count'})

,school_name,type,Student Count
0,Huang High School,District,2917
1,Figueroa High School,District,2949
2,Shelton High School,Charter,1761
3,Hernandez High School,District,4635
4,Griffin High School,Charter,1468
5,Wilson High School,Charter,2283
6,Cabrera High School,Charter,1858
7,Bailey High School,District,4976
8,Holden High School,Charter,427
9,Pena High School,Charter,962


In [567]:
#ANY CHANGE TO THIS CELL WARRENTS A RERUN OF THE DATAFRAME LOADING FOR SOME REASON
#Total school budget and per student spending
schoolSpending = schoolData[['school_name', 'budget']]
fullSummary = mergeDataFrames(fullSummary, schoolSpending, 'school_name')
#schoolSpending['Per Student Spending'] 
fullSummary['Per Student Spending'] = fullSummary['budget']/fullSummary['count']
fullSummary

,school_name,type,count,budget,Per Student Spending
0,Huang High School,District,2917,1910635,655.0
1,Figueroa High School,District,2949,1884411,639.0
2,Shelton High School,Charter,1761,1056600,600.0
3,Hernandez High School,District,4635,3022020,652.0
4,Griffin High School,Charter,1468,917500,625.0
5,Wilson High School,Charter,2283,1319574,578.0
6,Cabrera High School,Charter,1858,1081356,582.0
7,Bailey High School,District,4976,3124928,628.0
8,Holden High School,Charter,427,248087,581.0
9,Pena High School,Charter,962,585858,609.0


In [568]:
#Average test scores per school
schoolMathMean = fullData.groupby(['school_name'])['math_score'].mean()
schoolReadMean = fullData.groupby(['school_name'])['reading_score'].mean()

fullSummary = mergeDataFrames(fullSummary, schoolMathMean, 'school_name')
fullSummary = mergeDataFrames(fullSummary, schoolReadMean, 'school_name')
fullSummary.head()

,school_name,type,count,budget,Per Student Spending,math_score,reading_score
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020
2,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757


In [569]:
#Students per school passing math
mminScore = 70
mathPassData = fullData[(fullData['math_score'] >= mminScore)]
perSchoolMathPass = mathPassData[['school_name','math_score']].groupby(['school_name']).count()

fullSummary = mergeDataFrames(fullSummary, perSchoolMathPass, "school_name")

In [570]:
#Students per school passing reading
rminScore = 70
readPassData = fullData[(fullData['reading_score'] >= rminScore)]
perSchoolReadPass = mathPassData[['school_name','reading_score']].groupby(['school_name']).count()

fullSummary = mergeDataFrames(fullSummary, perSchoolReadPass, "school_name")

In [571]:
#Students per school passing both
rminScore = 70
mminScore = 70
bothPassData = fullData[(fullData["reading_score"] >= rminScore) & (fullData["math_score"] >= mminScore)]

perSchoolBothPass = bothPassData[['school_name', 'math_score']].groupby(['school_name']).count()
perSchoolBothPass.rename(columns={'math_score' : 'both_scores'}, inplace=True)

fullSummary = mergeDataFrames(fullSummary, perSchoolBothPass, 'school_name')

In [572]:
#Convert passing student count to passing rate
fullSummary['math_score_y'] = (fullSummary['math_score_y'] / fullSummary['count']) * 100
fullSummary['reading_score_y'] = (fullSummary['reading_score_y'] / fullSummary['count']) * 100
fullSummary['both_scores'] = (fullSummary['both_scores'] / fullSummary['count']) * 100

In [573]:
#Rename columns with correct description
fullSummary.rename(inplace = True, columns={
    'type' : 'Type',
    'count' : 'Count',
    'school_name' : 'School Name',
    'budget' : 'Budget',
    'math_score_x' : 'Average Math Score',
    'reading_score_x' : 'Average Reading Score',
    'math_score_y' : 'Percent Passing Math',
    'reading_score_y' : 'Percent Passing Reading',
    'both_scores' : 'Percent Passing Both'
})

fullSummary.set_index('School Name', drop=True, inplace=True)

#Apply formatting
fullSummary["Budget"] = fullSummary["Budget"].map("${:,.2f}".format)
fullSummary["Per Student Spending"] = fullSummary["Per Student Spending"].map("${:,.2f}".format)
#Display frame
fullSummary

,Type,Count,Budget,Per Student Spending,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
School Name,,,,,,,,,
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,65.683922,53.513884
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,65.988471,53.204476
Shelton High School,Charter,1761,"$1,056,600.00",$600.00,83.359455,83.725724,93.867121,93.867121,89.892107
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,66.752967,53.527508
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,93.392371,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,93.867718,90.582567
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,94.133477,91.334769
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,66.680064,54.642283
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,92.505855,89.227166


## Highest-Performing Schools (by % Overall Passing)

In [574]:
#Sort and display school summary dataframe by % overall passing column (desc)
topSchools = fullSummary.sort_values(by=['Percent Passing Both'], ascending=False)
topSchools.head(5)

,Type,Count,Budget,Per Student Spending,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,94.133477,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,93.272171,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,93.392371,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,93.867718,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,94.594595,90.540541


## Bottom Performing Schools (By % Overall Passing)

In [575]:
#Sort and display school summary dataframe by % overall passing column (asce)
botSchools = fullSummary.sort_values(by=['Percent Passing Both'], ascending=True)
botSchools.head(5)

,Type,Count,Budget,Per Student Spending,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
School Name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,66.366592,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,65.988471,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,65.683922,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,66.752967,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,66.057551,53.539172


## Math score by grade

In [576]:
#Seperate data by grade
ninethG = fullData[(fullData["grade"] == "9th")]
tenthG = fullData[(fullData["grade"] == "10th")]
elevenG = fullData[(fullData["grade"] == "11th")]
twelveG = fullData[(fullData["grade"] == "12th")]
#Group by 'school_name' and take the mean of the 'math_score' column for each.
nineMathMean = ninethG.groupby(['school_name'])['math_score'].mean()
nineMathMean.name = '9th'
tenMathMean = tenthG.groupby(['school_name'])['math_score'].mean()
tenMathMean.name = '10th'
elevenMathMean = elevenG.groupby(['school_name'])['math_score'].mean()
elevenMathMean.name = '11th'
twelveMathMean = twelveG.groupby(['school_name'])['math_score'].mean()
twelveMathMean.name = '12th'
#Combine each of the scores above into single DataFrame called 'gradeMathScores'
gradeMathScores = pd.concat([nineMathMean, tenMathMean, elevenMathMean, twelveMathMean], axis=1)
#Remove index title
gradeMathScores.index.name = None
#Display
gradeMathScores



,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading score by grade

In [577]:
#Seperate data by grade
ninethG = fullData[(fullData["grade"] == "9th")]
tenthG = fullData[(fullData["grade"] == "10th")]
elevenG = fullData[(fullData["grade"] == "11th")]
twelveG = fullData[(fullData["grade"] == "12th")]
#Group by 'school_name' and take the mean of the 'reading_score' column for each.
nineMathMean = ninethG.groupby(['school_name'])['reading_score'].mean()
nineMathMean.name = '9th'
tenMathMean = tenthG.groupby(['school_name'])['reading_score'].mean()
tenMathMean.name = '10th'
elevenMathMean = elevenG.groupby(['school_name'])['reading_score'].mean()
elevenMathMean.name = '11th'
twelveMathMean = twelveG.groupby(['school_name'])['reading_score'].mean()
twelveMathMean.name = '12th'
#Combine each of the scores above into single DataFrame called 'gradeReadScores'
gradeReadScores = pd.concat([nineMathMean, tenMathMean, elevenMathMean, twelveMathMean], axis=1)
#Remove index title
gradeReadScores.index.name = None
#Display
gradeReadScores

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by school spending NOT DONE

In [609]:
#Create bins
spendBins = [0, 1, 2, 3, 4]
labels = ["$0-$585", "$585-630", "$630-645", "$645-680"]
labels2 = ["1", "2", "3", "4"]

In [596]:
#Copy per school summary dataframe to work with
summaryCopy = fullSummary.copy()

In [610]:
#Use cut to put the schools in bins
summaryCopy["Spending Ranges (Per Student)"] = pd.cut(summaryCopy['Per Student Spending'], spendBins, labels=labels2)
summaryCopy

TypeError: '<' not supported between instances of 'int' and 'str'

In [ ]:
#Calculate Averages for the test score columns
spendingMathScores = schoolSpending.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spendingReadingScores = schoolSpending.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spendingMathPass = schoolSpending.groupby(["Spending Ranges (Per Student)"])["Percent Passing Math"].mean()
spendingReadPass = schoolSpending.groupby(["Spending Ranges (Per Student)"])["Percent Passing Reading"].mean()
spendingOverPass = schoolSpending.groupby(["Spending Ranges (Per Student)"])["Percent Overall Passing"].mean()

KeyError: 'Spending Ranges (Per Student)'

In [ ]:
#Assemble dataframe

## Scores by school size

In [583]:
#Create bins
sizeBins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

sizeCheck = summaryCopy

In [595]:
#Use cut to put the schools in bins
sizeCheck["School Size"] = pd.cut(sizeCheck['Count'], sizeBins, labels=labels)

In [589]:
#Calculate Averages for the test score columns
sizeMathScore = sizeCheck.groupby(["School Size"])["Average Math Score"].mean()
sizeReadScore = sizeCheck.groupby(["School Size"])["Average Reading Score"].mean()
sizeMathPass = sizeCheck.groupby(["School Size"])["Percent Passing Math"].mean()
sizeReadPass = sizeCheck.groupby(["School Size"])["Percent Passing Reading"].mean()
sizeBothPass = sizeCheck.groupby(["School Size"])["Percent Passing Both"].mean()

pandas.core.series.Series

In [590]:
#Assemble dataframe
sizeFrame = pd.concat([sizeMathScore, sizeReadScore, sizeMathPass, sizeReadPass, sizeBothPass], axis= 1)
sizeFrame

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,93.550225,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,93.599695,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,69.963361,58.286003


## Scores by school type

In [592]:
#Group per school summary by "School Type" and get averages for test score columns
avgMathScoreType = fullSummary.groupby(["Type"])["Average Math Score"].mean()
avgReadScoreType = fullSummary.groupby(["Type"])["Average Reading Score"].mean()
avgPctMathType = fullSummary.groupby(["Type"])["Percent Passing Math"].mean()
avgPctReadType = fullSummary.groupby(["Type"])["Percent Passing Reading"].mean()
avgPctBothType = fullSummary.groupby(["Type"])["Percent Passing Both"].mean()

In [594]:
#Assemble dataframe
typeFrame = pd.concat([avgMathScoreType, avgReadScoreType, avgPctMathType, avgPctReadType, avgPctBothType], axis = 1)
typeFrame

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Percent Passing Both
Type,,,,,
Charter,83.473852,83.896421,93.620830,93.620830,90.432244
District,76.956733,80.966636,66.548453,66.548453,53.672208
